In [1]:
# Import relevant packages
import pandas as pd
import numpy as np

import os

import geopandas
from shapely.geometry import Point

import re

import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.animation as ani
import matplotlib.font_manager as font_manager
import seaborn as sns

from PIL import Image

from scipy.stats import gaussian_kde
from scipy.stats import kstest
from scipy.stats import ks_2samp
from scipy import stats

from sklearn import linear_model
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.formula.api import ols

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
# from linearmodels import PanelOLS

proj_path = os.path.dirname(os.path.dirname(os.path.abspath('calwater_shortage.ipynb')))

dr_path = proj_path + r"/calwater_data_raw"
dc_path = proj_path + r"/calwater_data_clean"

calwater_output = proj_path + r"/calwater_output"

In [2]:
calwater_comp_imp = pd.read_csv(dc_path + r"/calwater_data.csv")

calwater_comp_imp.head()

,supplier_name,supplier_name2,population,tot_waterprod_exclag,share_resid,county,hydrologic_region,for_profit,date,ag_calc_waterprod,shortage_stage,calculated_r_gpcd,private,nonprofit,month,log_gpcd,public,owner
0,adelanto city of,adelanto city of,31765.0,124032544.0,0.710,San Bernardino,South Lahontan,0.0,2014-09-15,0.0,NaN,92.4110,NaN,0.0,9,4.526246,1.0,2.0
1,adelanto city of,adelanto city of,29855.0,94182732.0,0.730,San Bernardino,South Lahontan,0.0,2020-02-15,0.0,NaN,79.4107,NaN,0.0,2,4.374633,1.0,2.0
2,adelanto city of,adelanto city of,29545.0,92091927.0,0.750,San Bernardino,South Lahontan,0.0,2019-11-15,0.0,NaN,77.9251,NaN,0.0,11,4.355748,1.0,2.0
3,adelanto city of,adelanto city of,29707.0,105007792.0,0.655,San Bernardino,South Lahontan,0.0,2017-11-15,0.0,3,77.1761,NaN,0.0,11,4.346090,1.0,2.0
4,adelanto city of,adelanto city of,30044.0,151747574.0,0.600,San Bernardino,South Lahontan,0.0,2019-07-15,0.0,NaN,97.7583,NaN,0.0,7,4.582498,1.0,2.0


In [3]:
calwater_comp = calwater_comp_imp.copy()

calwater_comp['short_clean'] = calwater_comp['shortage_stage']

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('III', '3')
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('II', '2')
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace(' I ', '1')
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(I)$', '1', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('[Oo][Nn][Ee]', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('[Ff][Ii][Rr][Ss][Tt]', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('second', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('[Tt]wo', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('[Tt]hree', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('[Ff]our', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('[Zz]ero', '0', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Nn]\/[Aa]).*', '0', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Nn]1).*', '0', regex=True)
# calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Nn][Oo] [Ss]ta).*', '0', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^.*( no longer under).*', '0', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Nn]ormal).*', '0', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Vv]oluntary).*', '1', regex=True)

# Voluntary rationing: https://docs.cpuc.ca.gov/published/REPORT/99158.htm
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(Rul[er ]+14[.]+1).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(CPUC.* 14[.]+1).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([ ]*14[.]+1)$', '1', regex=True)

# Mild restriction on landscape watering: https://coolcalifornia.arb.ca.gov/tip/water-lg
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Jj]an[a-z ]+1[0]?,[ ]?2010).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Jj]an[a-z ]+2010)$', '1', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('([Ss][TtAaEe]+[gta]+e[gv]?)', 'Stage', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(1(st)? [Ss]tage).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(1(st)? [Ll]evel).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(2(nd)? [Ss]tage).*', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(3(rd)? [Ss]tage).*', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(3(rd)? [Ll]evel).*', '3', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(1(st)? of 4).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(2(nd)? of 4).*', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(3(rd)? of 4).*', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*(4(th)? of 4).*', '4', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(pahse 1).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Pp][g]?[Hh]?[Aa][Ss]+[Ee] IV).*', '4', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Pp][g]?[Hh]?[Aa][Ss]+[Ee] 4).*', '4', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Pp][g]?[Hh]?[Aa][Ss]+[Ee] [1iI]).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Pp][g]?[Hh]?[Aa][Ss]+[Ee] [2]).*', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Pp][g]?[Hh]?[Aa][Ss]+[Ee] [3]).*', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Pp][g]?[Hh]?[Aa][Ss]+[Ee] [5V]).*', '3', regex=True)


calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('([Ll][EeUe][Vv][Ee][Lld])', 'Level', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(Water (Supply )?Shortage.* Level)', 'Level', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^.*(pdated.[Ll]evel 1)', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^((Drought )?[Ll][Ee]ve[l]?[ ]?1.*[Mm]andatory).*', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^((Drought )?[Ll][Ee]ve[l]? [1I]).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^((Drought.*)?[Ll][Eue]ve[l]?[ ]?2).*', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ll][Ee]ve[l] 2 Water Alert).*', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^((Drought.*)?[Ll][Ee]ve[l]?[ ]?3).*', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ll][Ee]ve[l] 0).*', '0', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ll][Ee]ve[l] 1).*', '1', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?[O0]).*', '0', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?( No. )?[1iI]).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?2).*', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(Second( Stage)?)$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?3).*', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?4).*', '4', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?6).*', '6', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?7).*', '7', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Nn]o [Ss][Tt][Aa][Gg][Ee][ ]?)$', '0', regex=True)

# Assuming A-E stages similar to 1-5: https://codelibrary.amlegal.com/codes/beverlyhillsca/latest/beverlyhills_ca/0-0-0-11303
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?A).*', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?B).*', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?C).*', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?D).*', '4', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('.*([Ss][Tt][Aa][Gg][Ee][ ]?E).*', '5', regex=True)

# Stupid, nonsensical fucking color conversion: https://www.cvwd.com/ordinances
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Bb]lue)$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(Green)$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Yy]ello[w]?)$', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Oo]rang[we])$', '4', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(Red)$', '5', regex=True)

# Assuming loose restrictions
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*[Qq]ualif.*[Ss]ta[tg]eme.*)', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*[Qq]ualif.*[bB]elow.*)', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*See Note found [bB]elow.*)', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*CITY.*ORDINANCE.*)', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(Sta[mn]dard)$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(.*Resolution 2016-18.* Water).*$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(.*Water Use Efficiency Best Practices).*$', '1', regex=True) 
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(.*Following Federal Contingency Guidelines).*$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^Advisory$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(Conservation Rate)$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^yes$', '1', regex=True)

# Assuming tight restrictions
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*Rest[ricted ]+Use.*)', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*Use Restriction.*)', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*Temporary Water Restrictions.*)', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*Baseline Level[ -]+Mandatory.*)', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*[Mm]andat[ed ]+[Cc]omplian.*)', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(.*Significant Water Supply Shortage.*)', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(.*Resolution 2015-16.* Water).*$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(.*0 irrigation.*).*$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^Allocation$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^urgency measures$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Ss]ta[gt]e[s]?)( plan)?$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace("^(Governor['s]+ Proclamation)$", '2', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(1.?[ABCabc])$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(01)$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(1[- ]*[Mm]andatory)$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(1[- ]*in progress).*$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(1st)$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(Notice of.*Level 1)$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(2.?[ABCabc])$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(2.?[ Municipal Code]).*$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(2nd)$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(2.[5]?)$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(3.?[ABCabc])$', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(3rd)$', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(3.?)$', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(3.?[ABCabc]\:[ ]+Mand).*$', '3', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(4.?[ABCabc])$', '4', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(IV)$', '4', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(4.?[ABCabc]\: Mand).*$', '4', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(V)$', '5', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(V2)$', '7', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(0\, N1)$', '0', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(0\, Normal)$', '0', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('(["Pp]+ermanent.*)', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Mm]andatory.*)$', '1', regex=True)

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(Volunteer)$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Mm]inimal).*$', '1', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Mm]oderate).*$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Ss]ever[e]?ly [Rr]est).*$', '3', regex=True)\

calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^([Ee]xtra[or]+dinary).*$', '5', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^(.*[Ee]mergency).*$', '5', regex=True)

# Based on mean shortage
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^Level$', '2', regex=True)
calwater_comp['short_clean'] = calwater_comp['short_clean'].str.replace('^S$', '0', regex=True)

calwater_comp['short_clean2'] = ''

calwater_comp['short_clean2'] = calwater_comp.loc[calwater_comp['short_clean'].str.len()==1, 'short_clean']
calwater_comp['short_clean2'] = calwater_comp['short_clean2'].str.replace('^[ABCSVlo]$', '', regex=True)

calwater_comp.loc[calwater_comp['shortage_stage'].isnull(), 'short_clean2'] = '0'

calwater_comp['short_clean2'] = pd.to_numeric(calwater_comp['short_clean2'], errors='coerce')

print(calwater_comp['short_clean2'].count())

calwater_comp['short_clean2'].sort_values().unique()

calwater_comp.loc[calwater_comp['short_clean2'].isnull(), 'shortage_stage'].sort_values().unique()

calwater_comp.loc[(calwater_comp['short_clean2'].isnull()) & (calwater_comp_imp['shortage_stage'].isnull()==0), 'shortage_stage'].unique()

30177


array(['.0', 'restrictions rescinded', 'Non', 'non', 'No', 'NO',
       'CITY WATER CONSERVSTION', 'd2', 'll', 'A', 'B', 'volunteer',
       'mandatoy', '1440.7', 'Stag II', 'o', 'No WSCP invoked',
       'Water Watch', 'Water Supply Storage Alert',
       'Water Supply Shortage Alert', 'C', 'Level B', 'level c',
       'PERMANENT MANDATORY', 'PERMANENT', 'Rescinded June 2017',
       'rescinded June 2017', 'rescinded', 'Modified Normal', 'Severely',
       'no', 'No Stage yet- just permanent prohibitions',
       'NO Stage- Permanent Prohibitions',
       'NO Stage - Permanent Prohibitions',
       'no stage-just permanent prohibitions',
       'no stage yet- just general prohibitions', 'See Note', 'See note',
       "We haven't", 'In June, we still had not evoked our DCP.',
       'Water restricions', 'Water restrictions',
       'RESTRICTED OUTDOOR WATER USAGE', 'Outdoor irrigation',
       'Adequate Supply', 'Basic', 'basic', 'Modified',
       'Conservation Ordinance', 'Plan 2 of 

In [4]:
calwater_comp.loc[(calwater_comp['county'].isin(['Sacramento', 'Los Angeles', 'Kern'])) & 
                  (calwater_comp['short_clean2'].isnull()), ['county', 'shortage_stage', 'short_clean', 'log_gpcd']] #.drop_duplicates() #.head()

,county,shortage_stage,short_clean,log_gpcd


In [5]:
calwater_comp[calwater_comp['short_clean2']==7]

,supplier_name,supplier_name2,population,tot_waterprod_exclag,share_resid,county,hydrologic_region,for_profit,date,ag_calc_waterprod,shortage_stage,calculated_r_gpcd,private,nonprofit,month,log_gpcd,public,owner,short_clean,short_clean2
576,antioch city of,antioch city of,106455.0,277000000.0,0.7307,Contra Costa,San Joaquin River,0.0,2016-12-15,0.0,7,61.3326,NaN,0.0,12,4.116312,1.0,2.0,7,7.0
8406,el centro city of,el centro city of,46315.0,142540000.0,0.7000,Imperial,Colorado River,0.0,2019-01-15,0.0,7,69.4947,NaN,0.0,1,4.241250,1.0,2.0,7,7.0
8435,el centro city of,el centro city of,46315.0,120800000.0,0.7000,Imperial,Colorado River,0.0,2019-02-15,0.0,7,65.2057,NaN,0.0,2,4.177547,1.0,2.0,7,7.0
8441,el centro city of,el centro city of,46315.0,197470000.0,0.7000,Imperial,Colorado River,0.0,2019-04-15,0.0,7,99.4847,NaN,0.0,4,4.600004,1.0,2.0,7,7.0
8451,el centro city of,el centro city of,46315.0,171040000.0,0.7000,Imperial,Colorado River,0.0,2019-03-15,0.0,7,83.3897,NaN,0.0,3,4.423525,1.0,2.0,7,7.0
14058,la verne city of,la verne city of,32228.0,207567360.0,0.7138,Los Angeles,South Coast,0.0,2017-04-15,0.0,7,153.2430,NaN,0.0,4,5.032025,1.0,2.0,7,7.0
14068,la verne city of,la verne city of,32228.0,125778652.0,0.7574,Los Angeles,South Coast,0.0,2016-12-15,0.0,7,95.3536,NaN,0.0,12,4.557592,1.0,2.0,7,7.0
14069,la verne city of,la verne city of,32228.0,210500023.0,0.7258,Los Angeles,South Coast,0.0,2016-10-15,0.0,7,152.9230,NaN,0.0,10,5.029935,1.0,2.0,7,7.0
14073,la verne city of,la verne city of,32228.0,130992274.0,0.7574,Los Angeles,South Coast,0.0,2015-12-15,0.0,7,99.3061,NaN,0.0,12,4.598207,1.0,2.0,7,7.0
14075,la verne city of,la verne city of,32228.0,233309623.0,0.7448,Los Angeles,South Coast,0.0,2016-06-15,0.0,7,179.7290,NaN,0.0,6,5.191450,1.0,2.0,7,7.0


In [157]:
calwater_comp[calwater_comp['share_resid'].isnull()]

,supplier_name,supplier_name2,population,tot_waterprod_exclag,share_resid,county,hydrologic_region,for_profit,date,ag_calc_waterprod,shortage_stage,calculated_r_gpcd,private,nonprofit,month,log_gpcd,public,owner,short_clean,short_clean2
5154,camrosa water district,camrosa water district,35783.0,256119223.0,NaN,Ventura,South Coast,0.0,2020-06-15,31607589.0,0,166.7000,NaN,0.0,6,5.116196,1.0,2.0,0,0.0
5160,camrosa water district,camrosa water district,35783.0,200072777.0,NaN,Ventura,South Coast,0.0,2020-11-15,27371520.0,0,128.6740,NaN,0.0,11,4.857282,1.0,2.0,0,0.0
5161,camrosa water district,camrosa water district,35783.0,268501577.0,NaN,Ventura,South Coast,0.0,2020-08-15,49203566.0,0,165.5630,NaN,0.0,8,5.109352,1.0,2.0,0,0.0
5172,camrosa water district,camrosa water district,35783.0,131969828.0,NaN,Ventura,South Coast,0.0,2020-04-15,15640869.0,0,94.5495,NaN,0.0,4,4.549124,1.0,2.0,0,0.0
5174,camrosa water district,camrosa water district,35783.0,179869989.0,NaN,Ventura,South Coast,0.0,2020-12-15,25742263.0,0,112.9550,NaN,0.0,12,4.726990,1.0,2.0,0,0.0
5175,camrosa water district,camrosa water district,35783.0,146307292.0,NaN,Ventura,South Coast,0.0,2021-01-15,17595977.0,0,94.4630,NaN,0.0,1,4.548208,1.0,2.0,0,0.0
5176,camrosa water district,camrosa water district,35783.0,212780984.0,NaN,Ventura,South Coast,0.0,2020-05-15,29978331.0,0,132.3180,NaN,0.0,5,4.885208,1.0,2.0,0,0.0
5177,camrosa water district,camrosa water district,35783.0,245366127.0,NaN,Ventura,South Coast,0.0,2020-10-15,32259291.0,0,147.4710,NaN,0.0,10,4.993632,1.0,2.0,0,0.0
5178,camrosa water district,camrosa water district,35783.0,276647863.0,NaN,Ventura,South Coast,0.0,2020-09-15,34214400.0,0,173.9530,NaN,0.0,9,5.158785,1.0,2.0,0,0.0
5179,camrosa water district,camrosa water district,35783.0,293266286.0,NaN,Ventura,South Coast,0.0,2020-07-15,35191954.0,0,179.9090,NaN,0.0,7,5.192451,1.0,2.0,0,0.0


In [158]:
calwater_comp.loc[(calwater_comp['county'].isin(['San Bernardino', 'Los Angeles', 'Kern'])), ['short_clean2', 'log_gpcd']].groupby(['short_clean2']).mean()

,log_gpcd
short_clean2,
0.0,4.543458
1.0,4.487619
2.0,4.466138
3.0,4.525377
4.0,4.479565
5.0,4.922945
6.0,4.661070
7.0,4.704032


In [159]:

# calwater_comp.loc[calwater_comp['short_clean2'] > 1, 'crisis'] = 1
calwater_comp['crisis'] = calwater_comp['short_clean2'].isin([2,3,4,5,6,7]).astype(int)

calwater_comp.to_csv(dc_path + r"/calwater_data_shortage.csv")

In [163]:
calwater_comp[calwater_comp['supplier_name2']=='california city city of'].sort_values('date')

,supplier_name,supplier_name2,population,tot_waterprod_exclag,share_resid,county,hydrologic_region,for_profit,date,ag_calc_waterprod,...,calculated_r_gpcd,private,nonprofit,month,log_gpcd,public,owner,short_clean,short_clean2,crisis
2642,california city city of,california city city of,14120.0,136205897.0,0.6950,Kern,South Lahontan,0.0,2014-06-15,0.0,...,223.47300,NaN,0.0,6,5.409291,1.0,2.0,1,1.0,0
2666,california city city of,california city city of,14120.0,147284846.0,0.6940,Kern,South Lahontan,0.0,2014-07-15,0.0,...,233.51800,NaN,0.0,7,5.453259,1.0,2.0,1,1.0,0
2643,california city city of,california city city of,14120.0,120565029.0,0.6800,Kern,South Lahontan,0.0,2014-08-15,0.0,...,187.29800,NaN,0.0,8,5.232701,1.0,2.0,1,1.0,0
2667,california city city of,california city city of,14120.0,95148617.0,0.6700,Kern,South Lahontan,0.0,2014-09-15,0.0,...,150.49500,NaN,0.0,9,5.013930,1.0,2.0,1,1.0,0
2668,california city city of,california city city of,14120.0,110789486.0,0.7100,Kern,South Lahontan,0.0,2014-10-15,0.0,...,179.70500,NaN,0.0,10,5.191317,1.0,2.0,1,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,california city city of,california city city of,14120.0,95474469.0,0.5387,Kern,South Lahontan,0.0,2020-09-15,0.0,...,121.41700,NaN,0.0,9,4.799231,1.0,2.0,2,2.0,1
2596,california city city of,california city city of,14198.0,84721372.0,1.0000,Kern,South Lahontan,0.0,2020-10-15,0.0,...,129.73700,NaN,0.0,10,4.865509,1.0,2.0,2,2.0,1
2600,california city city of,california city city of,14198.0,59304960.0,1.0000,Kern,South Lahontan,0.0,2020-11-15,0.0,...,12.87910,NaN,0.0,11,2.555606,1.0,2.0,2,2.0,1
2662,california city city of,california city city of,14198.0,53765486.0,1.0000,Kern,South Lahontan,0.0,2020-12-15,0.0,...,8.79523,NaN,0.0,12,2.174210,1.0,2.0,2,2.0,1
